In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import re
sns.set_style('darkgrid')

In [ ]:
import sys
sys.path.append('../../pytpc')
import pytpc

# Calculation of Energy Loss formula for He-CO2 90/10

I found this by averaging together data from helium and CO2.

## ASTAR data

This data is from http://physics.nist.gov/PhysRefData/Star/Text/ASTAR.html. I believe it's experimental data, but it seems to be close enough either way.

In [ ]:
with open('../data/helium.txt') as f:
    hedata = []
    for line in f:
        elmts = line.strip().split()
        if any([re.match(r'\d*\.\d*E[+-]\d*', x) for x in elmts]):
            hedata.append([float(x) for x in elmts])
    hedata = pd.DataFrame(hedata, columns=('energy', 'dedx'))

In [ ]:
with open('../data/co2.txt') as f:
    codata = []
    for line in f:
        elmts = line.strip().split()
        if any([re.match(r'\d*\.\d*E[+-]\d*', x) for x in elmts]):
            codata.append([float(x) for x in elmts])
    codata = np.array(codata)
    codata = pd.DataFrame(codata, columns=('energy', 'dedx'))

The contribution from each gas is about the same, if we look just at stopping power in MeV / (g/cm^2).

In [ ]:
plt.plot(hedata.energy, hedata.dedx, label='ASTAR He')
plt.plot(codata.energy, codata.dedx, label='ASTAR CO2')
plt.loglog()
plt.legend()
plt.xlabel('Energy [MeV]')
plt.ylabel('Stopping Power [MeV/(g/cm^2)]');

However, the densities are quite different (by an order of magnitude!), so I'll find the weighted average using the density at each gas's partial pressure as the weighting factor.

## Weighted Average

In [ ]:
def density(pressure, molar_mass):
    """Find density in g/cm^3 from pressure in Torr 
    and molar mass in g/mol
    """
    return pressure / 760. * molar_mass / 24040.

hemass = 4.002
co2mass = 44.01
avgmass = hemass * 0.9 + co2mass * 0.1

hecodata = pd.DataFrame()
hecodata['energy'] = hedata.energy
hecodata['dedx'] = ((hedata.dedx * density(760*0.9, hemass) + codata.dedx * density(760*0.1, co2mass))
                    / density(760, avgmass))

Here's a plot of each gas's contribution to the stopping power (in MeV/m, this time). This is for a total pressure of 760 torr. It appears that the partial pressure of CO2 (10% of the total gas pressure) contributes nearly as much as the helium does (the other 90%). This implies that just using helium gas in the simulation causes an error of about a factor of 2 in the energy loss.

In [ ]:
plt.plot(hedata.energy, hedata.dedx * density(760*0.9, hemass) * 100, label='ASTAR He')
plt.plot(codata.energy, codata.dedx * density(760*0.1, co2mass) * 100, label='ASTAR CO2')
plt.plot(hecodata.energy, hecodata.dedx * density(760, avgmass) * 100, label='Weighted Mean')
plt.loglog()
plt.legend()
plt.ylabel('Stopping Power [MeV/m]')
plt.xlabel('Energy [MeV]');

## Fitting the weighted average

First I'll take a subset of the data, since we don't need it to fit well above a certain energy.

In [ ]:
fitdata = hecodata.copy()
fitdata.index = fitdata.energy
fitdata.drop('energy', 1, inplace=True)
fitdata = fitdata.loc[0.01:20.0]

In [ ]:
fitdata.plot()

In [ ]:
from scipy.optimize import curve_fit
from numpy import exp

The fit function is below:

$$
    f(x) = \frac{a}{x^b} \frac{1}{c + d\,x^{-e}} + f \exp(-g (x-h)^2)
$$

This came from Wolfi's Fortran program.

In [ ]:
def f(en, a, b, c, d, e, f, g, h):
    return a*(1./en**b)*(1./(c+d/(en**e))) + f*exp(-g*(en-h)**2)

Guess some parameters

In [ ]:
plt.plot(fitdata.index, [f(x, *[300., 0.18, 0.8, 1, 1.5, 0.5, 300., 1.5]) for x in fitdata.index])
plt.plot(fitdata.index, fitdata.dedx, '.')
plt.loglog()

Perform the actual fit

In [ ]:
popt, pcov = curve_fit(f, fitdata.index.values, fitdata.dedx.values, p0=[300., 0.18, 0.8, 1, 1.5, 0.5, 300., 1.5])
plt.plot(hecodata.energy, [f(x, *popt) for x in hecodata.energy])
plt.plot(hecodata.energy, hecodata.dedx, '.')
plt.loglog()
plt.xlim(0.001, 20)

In [ ]:
popt  # These are the optimal parameters

## SRIM CO2 Data

I didn't use this in the fit, but it's here for future reference

In [ ]:
with open('/Users/josh/Documents/Data/GasData/srim-co2.txt') as f:
    in_header = True
    while in_header:
        litems = f.readline().strip().split()
        if all([re.match(r'-+', x) for x in litems]) and len(litems) == 6:
            in_header = False
    
    srimdat = []
    while True:
        litems = f.readline().strip().split()
        if len(litems) == 1 and re.match(r'-+', litems[0]):
            break
        en, en_u, dedx_elec, dedx_nuc, *junk = litems
        if en_u == 'keV':
            en = float(en) * 1e-3
        elif en_u == 'MeV':
            en = float(en)
        else:
            raise ValueError('energy units?')
        
        dedx_elec = float(dedx_elec) * 1000
        dedx_nuc = float(dedx_nuc) * 1000
        
        srimdat.append([en, dedx_elec + dedx_nuc])
    
    srimdat = pd.DataFrame(srimdat, columns=['energy', 'dedx'])

In [ ]:
plt.plot(codata.energy, codata.dedx, label='ASTAR CO2')
plt.plot(srimdat.energy, srimdat.dedx, label='SRIM CO2')
plt.plot(hedata.energy, hedata.dedx, label='ASTAR He')
plt.plot(codata.energy, co2stop(codata.energy), label='Brendle CO2 fit')
plt.plot(simdat.energy, simdat.dedx, 'k--', label='Simulation Data')
plt.loglog();
plt.legend();
plt.xlabel('Energy [MeV]')
plt.ylabel('Stopping Power [MeV/(g/cm^2)]');
plt.savefig('/Users/josh/Desktop/stop.pdf')

# Simulated range

Use this to check the results

In [ ]:
pt = pytpc.Particle(4, 2, energy_per_particle=6/4.)
heco = pytpc.gases.HeCO2Gas(153.)
he = pytpc.gases.HeliumGas(153.)
ef = np.array([0, 0, 15e3])
bf = np.zeros(3)

simres = pytpc.track(pt, he, ef, bf)

pt.energy = 6
pt.position = np.array((0, 0, 0))
simres2 = pytpc.track(pt, heco, ef, bf)

In [ ]:
plt.plot(simres['pos'][:, 2], simres['en']*pt.mass_num)
plt.plot(simres2['pos'][:, 2], simres2['en']*pt.mass_num)